In [1]:
# Создаем метод
class Parrot:
    
    # атрибуты экземпляра
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    # метод экземпляра
    def sing(self, song):
        return "{} поет {}".format(self.name, song)

    def dance(self):
        return "{} танцует".format(self.name)

# создаем экземпляр класса
kesha = Parrot("Кеша", 10)

# вызываем методы экземпляра
print(kesha.sing("песенки"))
print(kesha.dance())

Кеша поет песенки
Кеша танцует


In [2]:
class Bird:
    
    def __init__(self):
        print("Птица готова")

    def whoisThis(self):
        print("Птица")

    def swim(self):
        print("Плывет быстрее")

# дочерний класс
class Penguin(Bird):

    def __init__(self):
        # вызов функции super() 
        super().__init__()
        print("Пингвин готов")

    def whoisThis(self):
        print("Пингвин")

    def run(self):
        print("Бежит быстрее")

peggy = Penguin()
peggy.whoisThis()
peggy.swim()
peggy.run()

Птица готова
Пингвин готов
Пингвин
Плывет быстрее
Бежит быстрее


In [3]:
class Computer:

    def __init__(self):
        self.__maxprice = 900

    def sell(self):
        print("Цена продажи: {}".format(self.__maxprice))

    def setMaxPrice(self, price):
        self.__maxprice = price

c = Computer()
c.sell()

# изменение цены
c.__maxprice = 1000
c.sell()

# используем функцию изменения цены
c.setMaxPrice(1000)
c.sell()

Цена продажи: 900
Цена продажи: 900
Цена продажи: 1000


In [4]:
# Используем полиморфизм
class Parrot:

    def fly(self):
        print("Попугай умеет летать")
    
    def swim(self):
        print("Попугай не умеет плавать")

class Penguin:

    def fly(self):
        print("Пингвин не умеет летать")
    
    def swim(self):
        print("Пингвин умеет плавать")

# общий интерфейс 
#def flying_test(bird):
#    bird.fly()

# создаем экземпляров класса
kesha = Parrot()
peggy = Penguin()

# передача объектов в качестве аргумента
flying_test(kesha)
flying_test(peggy)

NameError: name 'flying_test' is not defined

## Переписываем скрипт

In [1]:
import sys
import os
import pandas as pd
from Bio import SeqIO

In [2]:
class SAM():
    def __init__(self, names, refs, start, cigar, read):
        self.names = names
        self.refs = refs
        self.start = start
        self.cigar = cigar
        self.read = read       

In [3]:
def Ref(Ref_name):
    with open('All_refs.fasta', 'r') as refs:
        line = refs.readline()
        ref_all = []
        while line:
            if Ref_name in line:# Если название референса из атрибутов совпадает с названием строки в фаста файле с референсами, следующую строку записываем как референс
                line = refs.readline()
                while not line.startswith(">"):
                    ref = list(line)[:-1]
                    ref_all = ref_all + ref
                    line = refs.readline()
                    if line == "":
                        break
            line = refs.readline()  
    return ref_all

In [4]:
class READ:
    '''Объект - рид из sam-файла. 
       Атрибуты: Имя рида, имя референса, старт, сторка сигар, сам рид
       Методы: Получить рид как он есть при выравнивании'''
    def __init__(self, name, start, CIGAR, read): # задаем объект, где как атрибуты будут имя рида, флаг и тд
        self.name = name
        self.start = start
        self.CIGAR = CIGAR
        self.read = read
    
    def ReadySeq(self): # Получаем рид как он будет при выравнивании.
        '''For input the self object, for output the seq is ready for comparing with reference'''
        cigar = list(self.CIGAR.strip()) #сделали список
        # для начала нужно сигар разбить на блоки и переделать цифры в инт. По идее олжно получиться int(61) str(S) int(37) str(M)...
        # Каждый символ пытаюсь перевести в инт, обработка исключений на буквы
        for i in range(len(cigar)): #пробуем каждый символ списка сделать интом
            try:
                cigar[i] = int(cigar[i])
            except ValueError:
                cigar[i] = cigar[i]
                # чтобы не использовать исключения, можно юзать ord chr, обращаясь к символам в кодировке, по времени может быть быстрее

        # Потом объединяю цифры в одно число.
        cigar_new = []
        for i in range(len(cigar)): # если символ - строка - записываем его отдельно
            if type(cigar[i]) == str:
                cigar_new.append(cigar[i])                                    
            try:
                if type(cigar[i]) == type(cigar[i+1]) == type(cigar[i+2]) : # если три символа подряд это инт, объединяем их
                    cigar_new.append(int(str(cigar[i])+str(cigar[i+1])+str(cigar[i+2])))
                if type(cigar[i]) == type(cigar[i+1]) and type(cigar[i]) != type(cigar[i+2]) and type(cigar[i-1]) ==str:  # если два символа подряд одного типа (такое может быть только с инт), объединяем их  
                    cigar_new.append(int(str(cigar[i])+str(cigar[i+1])))
                if type(cigar[i+1]) == str and type(cigar[i-1]) == str: # если один инт - тоже записываем его
                    cigar_new.append(cigar[i])
            except IndexError: # это нужно на случай окончания строки, list index out of range
                continue
                
        # удобнее сделать словарь
        numbers = cigar_new[::2] # список с числом нуклеотидов
        clipping_type = [] # список с типом совпадений
        
        for i in range(1, len(cigar_new), 2): 
            clipping_type.append(cigar_new[i]);

        cigar_dict = [] # делаем словарь
        for i in range(len(numbers)):
            cigar_dict.append((numbers[i],clipping_type[i]))
            
        # Теперь прогоняем сам seq по строке cigar, для каждой буквы свое действие
        nucl_number = 0 # сразу столбец из data
        seq = self.read
        for e in cigar_dict:
            if e[1] == 'S':
                seq = seq[:nucl_number] + 'N' * e[0] + seq[nucl_number + e[0]:]
            if e[1] == 'D':
                seq = seq[:nucl_number] + ' ' * e[0] + seq[nucl_number:]
            if e[1] == 'H':
                continue # это пока, может быть hard clipping не вырезается
            if e[1] == 'I':
                seq = seq[:nucl_number] + 'N' * e[0] + seq[nucl_number + e[0]:]
            nucl_number += e[0]
        seq = seq.replace('N', '') 
        new_seq = (self.start-1) * ' ' + seq #Тут может быть ошибка, мб не надо -1

        return new_seq

In [5]:
class ALIGN():
    '''Объект -  отдельное выравнивание.
    Атрибуты: референс по буквам, набор выровненных ридов, имена ридов, строки сигар для ридов'''
    def __init__(self, ref, reads, reads_names, reads_cigar):
        self.ref = ref
        self.reads = reads
        self.reads_names = reads_names
        self.reads_cigar = reads_cigar
        
    # Функция анализа покрытия выравнивания
    def RefCover(self):
        '''Делает словарь покрытий по конкретному референсу и сборке.
        На вход: Референс по буквам и Список ридов, 
        на выходе кортеж (список словарей с покрытием в процнтах для каждого нуклеотида в референсе, абсолютное покрытие этого нуклеотида)'''
        
        #Создаем список словарей, по длине референса, для каждого элемента 4 варианта нуклеотидов
        abs_cover_for_all_nucl = [] # большой словарь для всех нуклеотидов, по всей длине референса, абсолютные значения
        percent_cover_for_all_nucl = [] # большой словарь для всех нуклеотидов, по всей длине референса, процентные значения
        for i in range(len(self.ref) + 1):
            abs_cover_dict = {} # для каждого нуклеотида
            abs_cover_dict['A'] = 0
            abs_cover_dict['C'] = 0
            abs_cover_dict['G'] = 0
            abs_cover_dict['T']= 0
            abs_cover_for_all_nucl.append(abs_cover_dict)
            per_cover_dict = {} # для каждого нуклеотида
            per_cover_dict['A'] = 0
            per_cover_dict['C'] = 0
            per_cover_dict['G'] = 0
            per_cover_dict['T'] = 0
            percent_cover_for_all_nucl.append(per_cover_dict)
            
        # abs_cover_for_all_nucl - абсолютное покртыие в нуклеотидах
        # percent_cover_for_all_nucl - процентное покрытие в нуклеотидах

        # заполняем типами нуклеотидов, в абсолютных значениях
        for read in self.reads:
            for nucl_number, nucl in enumerate(read):
                #print(nucl, nucl_number)
                if nucl == 'A':
                    abs_cover_for_all_nucl[nucl_number]['A']  += 1
                if nucl == 'T':
                    abs_cover_for_all_nucl[nucl_number]['T']  += 1
                if nucl == 'G':
                    abs_cover_for_all_nucl[nucl_number]['G']  += 1
                if nucl == 'C':
                    abs_cover_for_all_nucl[nucl_number]['C']  += 1

        cover_abs = [] #абсолютное покрытие референса, без учета типа нуклеотида
        for nucl_number, nucl in enumerate(abs_cover_for_all_nucl): #проходимся по номеру элемента(нуклеотиду), и его значениям(словарю с типом и количеством)
            cover = 0 # общее покрытие определенного нуклеотида
            for key, value in nucl.items(): #Раскрываем  словарь, тип нуклеотидиа и количество 
                cover += value
            # посчитали общее абсолютное покрытие нуклеотида - cover
            # Далее заполняем процент различных типов, для каждой позиции в референсе
            for key, value in nucl.items():    
                if key == 'A':
                    if cover == 0:
                        percent_cover_for_all_nucl[nucl_number][key] =  0
                    else:
                        percent_cover_for_all_nucl[nucl_number][key] =  value * 100 / cover
                if key == 'T':
                    if cover == 0:
                        percent_cover_for_all_nucl[nucl_number][key] =  0
                    else:
                        percent_cover_for_all_nucl[nucl_number][key] =  value * 100 / cover
                if key == 'G':
                    if cover == 0:
                        percent_cover_for_all_nucl[nucl_number][key] =  0
                    else:
                        percent_cover_for_all_nucl[nucl_number][key] =  value * 100 / cover
                if key == 'C':
                    if cover == 0:
                        percent_cover_for_all_nucl[nucl_number][key] =  0
                    else:
                        percent_cover_for_all_nucl[nucl_number][key] =  value * 100 / cover
            cover_abs.append(cover)    
        return percent_cover_for_all_nucl, cover_abs
    
    # Функция с описанием замен сборки (выравнивания)
    def AllChanges(self):
        '''Смотрим на все существующие замены в этой сборке для этой хромосомы.
            На вход берем референс и риды сборки. На выходе получаем список списков, для каждого рида свой список.
            Внутри два элемента, номер рида и номер нуклеотида с типом замены'''
        changing = []
        for nucl_number, nucl in enumerate(self.ref): # для всех нуклеотидов референса
            for seq_number, seq in enumerate(self.reads): # для каждого рида сборки
                #print(seq[nucl_number])
                try:
                    if seq[nucl_number] != ' ': # если нуклеотид рида существует в этой позиции
                        if nucl != seq[nucl_number]: # если нуклеотид референса не равен нуклеотиду рида
                            changing.append([self.reads_names[seq_number], 
                                             '{0:s}'.format(seq[nucl_number], nucl_number +1), len(seq.replace(' ', ''))])                 
                except IndexError:
                    continue
        return changing

    def ReadWithChanges(self, AllChanges):
        '''Считаем количество ридов с заменами и считаем число замен в риде.
            На вход подаем список списков из функции AllChanges.
            На выход список кортежей с номером рида и числом замен в этом риде'''
        read_with_change = [] # Риды с заменами
        for e in AllChanges: # Для каждого элемента из общего файла существующих замен:
            read_with_change.append(e[0]) # Добавляем сюда номер рида

        change = [] 
        for read in read_with_change:
            change.append(('{0:s}'.format(read), read_with_change.count(read))) # Считаем сколько раз этот рид встретился в наборе всех замен
        read_with_change = list(set(change)) # Берем сет от этого набора:
        return read_with_change
    
    
    def Cleaning(self, ReadWithChanges, AllChanges, RefCover):
        candidate_for_remove = [] # список кандидатов на удаление
        for read_tuple in ReadWithChanges: # для всех ридов с заменами:
            for i in range(len(AllChanges)): # пройдемся по списку замен
                if read_tuple[0] == AllChanges[i][0]: # если рид с заменой в списке всех изменений, берем его длину
                    if read_tuple[1]*100/AllChanges[i][-1] >= 10: # Более скольки процентов ридов
                        candidate_for_remove.append(read_tuple)

        count =[]
        for read_number, read in enumerate(candidate_for_remove): # в каждом риде с большим числом замен
            k = 0 # Счетчик редких замен
            for i, e in enumerate(AllChanges): # иду по списку всех возможных замен
                if read[0] == e[0]: # если совпадает название рида с ошибкой и название кандидата на удаление
                    for cover_i, cover in enumerate(RefCover[0]): # смотрим словарь покрытия
                        for key, value in cover.items(): # key - тип нуклеотида с заменой и его номер, value - процент покрытия для этого типа замены
                            if key == e[1]: # когда нуклеотид совпадает с нуклеотидом из списка замен
                                if value * RefCover[1][cover_i] / max(RefCover[1]) < 40: # для скольки процентов ридов характнрна такая замена
                                    #print('Процент замен в риде, относительно его длины:', read[1],'\n', 'Информация о риде и замене:',e, '\n','Процент такой замены в целом:',value)
                                    k+=1
            if k !=0:
                count.append((read,k)) # сколько в риде всего замен, и сколько из них "Редкие"
        reads_for_remove = []
        for i in count:
            if i[1] / i[0][1] > 0.4: #если относительно всех замен рида более 40% из них редкие,
                reads_for_remove.append(i[0][0])
                
        # добавить еще риды с инсерцией и делецией, если в риде более 10 процентов от длины пропущено/добавлено
        #for i in range(len(self.reads_names)):
        #    if self.reads_cigar[i].count('I') >= 5 or self.reads_cigar[i].count('D') >= 5:
         #       reads_for_remove.append(self.reads_names[i])
            

        return reads_for_remove


In [ ]:
curpath = os.path.abspath(os.path.curdir) # зафиксируем папку    
files_sam_list = [x for x in os.listdir(curpath) if x.startswith("cart_")] # endswith - кончается на (можно так сделать для начала)

for sam_file in files_sam_list:

    print("Идет обработка файла: ", sam_file)
    data = pd.read_csv(sam_file, sep = '\t',header=None, usecols=[0,3,9,5,1,2], index_col = False)
    data.columns = ['Название рида','Flag','Ref','Начало рида', 'CIGAR','Рид']
    data = data.loc[data['Flag'] != 2048] # чистка по флагам
    data = data.loc[data['Flag'] != 2064]
    data = data.reset_index(drop=True)
    data = SAM(data['Название рида'], data['Ref'], data['Начало рида'], data['CIGAR'], data['Рид'])
    refs_in_sam = set(data.refs) # в sam файле смотрим сколько всего референсов.
    print("Всего референсов:",len(refs_in_sam))
    for i, ref in enumerate(refs_in_sam): # открываем один из референсов: (весь алгоритм был расчитан на такую сборку)
        reads = [] # в списке будут лежать риды для этого конкретного референса
        Names = []
        refs = Ref(ref)
        #print('номер референса - ', i, "Имя рефа", ref)
        for i in range(len(data.refs)):
            if data.refs[i].encode() == ref.encode():
                reads.append(READ(data.names.iloc[i], data.start.iloc[i], data.cigar.iloc[i], data.read.iloc[i]).ReadySeq())
                Names.append(data.names.iloc[i])
        print('Очистка сборки на референс: ', ref)
        # зададим выравниевание отдельным объектом
        align = ALIGN(refs, reads, data.names, data.cigar)
        bad_reads = align.Cleaning(align.ReadWithChanges(align.AllChanges()), align.AllChanges(), align.RefCover())   
        good_reads = []
        for read in Names:
            if read not in bad_reads:
                good_reads.append(read + '\t')
        print("Число плохих ридов",len(bad_reads), ", Число хороших ридов", len(good_reads))
        with open("Clear_{0:s}".format(sam_file), 'a') as output:
            with open(sam_file, "r") as file:
                for lines in file:
                    for read in good_reads:
                        if read in lines:
                            if ref in lines:
                                #print(read, lines)
                                output.write(lines)

Идет обработка файла:  cart_Non_adapters_IonXpress_010_rawlib.basecaller.fastq.sam
Всего референсов: 707
Очистка сборки на референс:  UCE776_Claster_0_Contig1
Число плохих ридов 0 , Число хороших ридов 1288
Очистка сборки на референс:  UCE122_Claster_1_Contig1
Число плохих ридов 0 , Число хороших ридов 288
Очистка сборки на референс:  UCE332_Claster_1_Contig1
Число плохих ридов 0 , Число хороших ридов 908
Очистка сборки на референс:  UCE21_Claster_1_Contig2
Число плохих ридов 0 , Число хороших ридов 26
Очистка сборки на референс:  UCE855_Claster_0_Contig1
Число плохих ридов 0 , Число хороших ридов 735
Очистка сборки на референс:  UCE26_Claster_0_Contig1
Число плохих ридов 0 , Число хороших ридов 1490
Очистка сборки на референс:  UCE1324_Claster_0_Contig1
Число плохих ридов 0 , Число хороших ридов 1478
Очистка сборки на референс:  UCE1266_Claster_0_Contig3
Число плохих ридов 0 , Число хороших ридов 1238
Очистка сборки на референс:  UCE693_Claster_0_Contig1
Число плохих ридов 0 , Число х